## Neural Network Model

In [1]:
import numpy as np
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

In [2]:
# Load Preprocessed Data
X_train = np.load(r"../data/processed/X_train.npy")
X_test  = np.load(r"../data/processed/X_test.npy")
y_train = np.load(r"../data/processed/y_train.npy")
y_test  = np.load(r"../data/processed/y_test.npy")

In [ ]:
# Initialize Neural Network
nn_model = MLPClassifier(random_state=42, max_iter=500)

In [ ]:
# Hyperparameter Tuning 
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive']
}
grid_search = GridSearchCV(nn_model, param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)